# Applied Data Visualization – Homework 4
*https://www.dataviscourse.net/2024-applied/*


In this homework we will create custom maps using GeoPandas and Contextily. 

## Your Info and Submission Instructions

* *First name: Logan*
* *Last name: Correa*
* *Email: u1094034@umail.utah.edu*
* *UID: u1094034*



For your submission, please do the following things: 
* **rename the file to `hw4_lastname.ipynb`**
* **include all files that you need to run the homework, including the data file provided** 
* **don't use absolute paths, but use a relative path to the same directory for referencing data**

## Dependencies

You will need to install the geopandas and the contextily package for this homework. If you're using our recommended python setup using Anaconda, the following commands will install these for you: 

`conda install -c conda-forge geopandas`  
`conda install contextily --channel conda-forge`   
`conda install fiona --channel conda-forge`  

You can also install these packages via the anaconda navigator, but you'll first have to add the `conda-forge` channel. That channel contains a larger set of more experimental packages. 

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Need for this homework
import geopandas as gpd
import contextily as cx

plt.style.use('default')
# This next line tells jupyter to render the images inline
%matplotlib inline
import matplotlib_inline
# This renders your figures as vector graphics AND gives you an option to download a PDF too
matplotlib_inline.backend_inline.set_matplotlib_formats('svg', 'pdf')

# Part 1: Avalanche Map

By this point you are very familiar with the Utah Avalanche Center data set and have answered many questions based on the data. But where do avalanches happen, exactly? Thankfully the data set also includes latitude and longitude values. We can use those to plot the exact locations on a map.

Create 2 maps: 
1. one with a **topographic-style streetmap** and just the avalanche locations marked in a contrasting color, and
2. one with a **neutral-style streetmap** and avalanche locations colored by the `Region` variable from the data set.

Hints:
- There are a couple of outlier data points clearly entered by error. Filter out all observations that are *not* in the bounding box of 36 & 42 degrees of latitude and -114 & -108 degrees of longitude.
- Map providers we used in the example are the `OpenTopoMap` and `CartoDB.Positron` but feel free to use any other similar types (see Contextily documentation [here](https://contextily.readthedocs.io/en/latest/providers_deepdive.html)).

Your result should look somewhat like this: 

![Two maps of Utah showing locations of avalanches.](avalanche_map.svg)

In [51]:
# Read in data
avy_df = pd.read_csv('./avalanches.csv')
#display(avy_df.head())
#display(avy_df['Coordinates'].head())

# Split the coordinates into two columns
avy_df[['Latitude', 'Longitude']] = avy_df['Coordinates'].str.split(',', expand=True)

# Convert the columns to numeric
avy_df['Latitude'] = pd.to_numeric(avy_df['Latitude'], errors='coerce')
avy_df['Longitude'] = pd.to_numeric(avy_df['Longitude'], errors='coerce')

# filter lat/long outliers
avy_filtered = avy_df[
    (avy_df['Latitude'] >= 36) & 
    (avy_df['Latitude'] <= 42) & 
    (avy_df['Longitude'] >= -114) & 
    (avy_df['Longitude'] <= -108)
]


In [58]:
from shapely.geometry import Point

# Convert latitude and longitude into Points (geometry)
geometry = [Point(xy) for xy in zip(avy_filtered['Longitude'], avy_filtered['Latitude'])]

# Create a GeoDataFrame with the Points
gdf = gpd.GeoDataFrame(avy_filtered, geometry=geometry)

# Set the coordinate reference system to WGS 84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

# Reproject to Web Mercator (EPSG:3857) to match the basemap projection
gdf = gdf.to_crs(epsg=3857)

# Plot the avalanche locations on an OpenTopoMap basemap
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the points (avalanche locations) in red
gdf.plot(ax=ax, color='blue', markersize=50, alpha=0.25)

# Add the OpenTopoMap basemap
cx.add_basemap(ax, source=cx.providers.OpenTopoMap)

# Remove X and Y axis
ax.set_axis_off()

# Set title and show the map
ax.set_title("Avalanche Locations on OpenTopoMap", fontsize=15)
plt.show()

### Analysis

What are the pros and cons of the two maps you created?

# Part 2: World Map Choropleth

Use the [IMF Climate Change data](https://climatedata.imf.org/pages/climatechange-data) provided in the `Annual_Surface_Temperature_Change.csv` file to make a choropleth map of the world, with each country colored by their latest observations (`F2022` variable). We provided a JSON file `world.json` with country geometries.

Hints:
- Our map is using the Robinson projection (`ESRI:54030`).
- You will need to `.merge()` the IMF DataFrame with the polygons GeoDataFrame.
- Make sure to use a diverging colormap centered at zero degrees. One way to center it is to set symmetric `vmin` and `vmax` parameters in GeoPandas's `.plot()` method.

Here's what your result should look like: 

![A choropleth map of tempurature changes across the world.](temperatures.svg)

In [53]:
# Read in geometries
world = gpd.read_file('./world.json')

# Read in IMF data
imf_df = pd.read_csv('./Annual_Surface_Temperature_Change.csv')

### your code here

## Bonus: Interactive Avalance Map

As you saw, there are a lot of avalanches in small regions. For this bonus question, create an interactive, zoomable version of the avalance map. 

Full credit here will require a nice design and implementation. 

# Grading Scheme

* Part 1: 5 points
* Part 2: 5 points
* Part 3: 2 points